### 先頭の単語だけ固定し、2番目の単語を総当りして探索

In [1]:
ex_num = "004"

In [2]:
import os
# 環境によって変更
os.chdir('/home/jovyan/work/notebook')

# path = "../../output"
path = "../model"

In [3]:
import numpy as np
import pandas as pd
import os
import time
import csv

from tqdm.notebook import tqdm

import random
from typing import List, Callable
import itertools, math
from typing import List, Tuple

from utils import PerplexityCalculator

### データ読み込み

In [4]:
df = pd.read_csv("../input/sample_submission.csv")

In [5]:
sub_df = df.copy()
sub_df["best_value"] = 0

In [6]:
df

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


### 関数

In [28]:
def simulated_annealing(
    scorer,
    pattern_df,
    max_iterations: int = 1000,
    BATCH_SIZE: int = 32,
    initial_temperature: float = 100.0,
    cooling_rate: float = 0.99
) -> Tuple[List[str], float]:
    """
    焼きなまし法で最適な順序を探索する。

    Args:
        scorer: 評価関数を提供するオブジェクト。
        pattern_df: パターンのデータフレーム。
        max_iterations (int): 最大反復回数。
        BATCH_SIZE (int): バッチサイズ。
        initial_temperature (float): 初期温度。
        cooling_rate (float): 冷却率。

    Returns:
        solution_df: 最適な順序のデータフレーム。
    """
    
    solution_df = pattern_df.copy()
    solution_df["score"] = solution_df["score"].astype(float)
    
    words_len = len(solution_df.iloc[0]["text"].split())
    pattern_len = len(solution_df)
    
    temperature = initial_temperature
    
    perms = []
    best_score = 10000
    global_best_sequence = ""
    
    for iteration in tqdm(range(max_iterations)):
        # 1行ずつ処理する
        for idx in range(pattern_len):
            current_sequence = solution_df.iloc[idx]["text"].split()
            
            neighbor_sequence = current_sequence[:]
            
            # 最初と最後の単語を固定し、ランダムに交換する
            i, j = random.sample(range(1, words_len), 2)
            neighbor_sequence[i], neighbor_sequence[j] = neighbor_sequence[j], neighbor_sequence[i]
            
            perms.append(" ".join(neighbor_sequence))
        
        # 評価値をまとめて計算
        neighbor_values = scorer.get_perplexity(perms, batch_size=min(pattern_len, BATCH_SIZE))

        # print(f"neighbor_values = {neighbor_values}")
        # 焼きなましによる評価値更新
        for idx in range(pattern_len):
            perm = perms[idx]
            neighbor_value = neighbor_values[idx]
            current_value = solution_df.iloc[idx]["score"]
            # print(f"current_value = {current_value}, neighbor_value = {neighbor_value}")
            
            # 確率的に次の状態を受け入れるかを決定
            delta = neighbor_value - current_value
            acceptance_probability = math.exp(-delta / temperature) if delta > 0 else 1.0

            # ベスト解を更新
            if (random.random() < acceptance_probability) or (neighbor_value < current_value):
                current_sequence = perm.split(" ")
                current_value = neighbor_value
                
                solution_df.loc[solution_df.index[idx], "text"] = perm
                solution_df.loc[solution_df.index[idx], "score"] = current_value
                start_word = solution_df.iloc[idx]["start"]
                # print(f"start_word = {start_word}, end_word = {end_word}, New best = {current_value} with '{current_sequence}'")

            if current_value < best_score:
                best_score = current_value
                global_best_sequence = perm
                print(f"New best = {best_score} with '{perm}'")
                
        # バッチリセット
        perms = []

        # 温度を更新
        if iteration % 10 == 0:
            temperature *= cooling_rate
                    
        # 温度が非常に低くなったら終了
        if temperature < 1e-5:
            break

        # iteration100回ごとに現在の温度を表示
        if iteration % 100 == 0:
            print(f"Iteration {iteration}, Temperature = {temperature}")
            
    return solution_df, best_score, global_best_sequence
     

In [29]:
def search_optimal_permutation(id: int, scorer, max_iterations: int = 1000, BATCH_SIZE=64):
    
    df = pd.read_csv("../input/sub_20241218_01.csv")
    
    # 文字列を単語に分割
    words = df.loc[id,"text"].split()
    
    # データをタプルのリストとして準備
    pattern_data = []
    
    words_len = len(words)
    for i in range(1, words_len):
        new_words = words[:]
        if i != 1:
            new_words[i], new_words[1] = new_words[1], new_words[i]
        
        # new_words[1]意外の単語をシャッフル
        random.shuffle(new_words[1:])
        
        pattern_data.append((new_words[0], " ".join(new_words), 10000))
            
    pattern_df = pd.DataFrame(pattern_data, columns=["start", "text", "score"])        

    start = time.time()
    
    # SAで最適な順序を探索
    solutions_df, best_value, best_sequence = simulated_annealing(
        scorer,
        pattern_df,
        max_iterations=max_iterations,
        BATCH_SIZE=BATCH_SIZE,
        initial_temperature=50.0,
        cooling_rate=0.99
    )
    
    # # optimized_dfのscore列の値が最小の行を取得
    # best_sequence = solutions_df.loc[solutions_df["score"].idxmin(), "text"]
    # best_value = solutions_df["score"].min()
    
    print(f"{id}th sample: {best_sequence}")
    print(f"Elapsed time: {time.time() - start:.2f} sec")
    print(f"Best value: {best_value}")
    
    return best_sequence, best_value, solutions_df


In [23]:
# 文字列型にのみダブルクォーテーションを付ける関数
def add_quotes_to_strings(value):
    if isinstance(value, str):  # 文字列型の場合
        return f'"{value}"'
    return value  # それ以外はそのまま

In [10]:
# LOAD GEMMA SCORER
scorer = PerplexityCalculator(f'{path}/gemma_2_9b')

cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [11]:
# scorer.clear_gpu_memory()

In [30]:
for i in [1, 2]:
    
    print(f"Processing {i}th sample")
    best_sequence, best_value, solutions_df = search_optimal_permutation(
        id=i,
        scorer=scorer,
        max_iterations=2**4,
        BATCH_SIZE=16
    )
    
    print(best_sequence)
    sub_df.loc[i, "best_value"] = best_value
    sub_df.loc[i, "text"] = best_sequence
    
    display(solutions_df)
    solutions_df.to_csv(f"{path}/out/solutions_{i}_{ex_num}.csv", index=False, header=True)



Processing 1th sample


  0%|          | 0/16 [00:00<?, ?it/s]

New best = 805.4541259733185 with 'reindeer mistletoe elf gingerbread ornament advent chimney fireplace family give sleep drive walk jump bake laugh night and the scrooge'
Iteration 0, Temperature = 49.5
New best = 771.577792875611 with 'reindeer mistletoe elf gingerbread ornament advent chimney fireplace laugh give sleep drive walk jump bake family night and the scrooge'
1th sample: reindeer mistletoe elf gingerbread ornament advent chimney fireplace laugh give sleep drive walk jump bake family night and the scrooge
Elapsed time: 32.06 sec
Best value: 771.577792875611
reindeer mistletoe elf gingerbread ornament advent chimney fireplace laugh give sleep drive walk jump bake family night and the scrooge


,start,text,score
0,reindeer,reindeer mistletoe family gingerbread ornament...,874.312182
1,reindeer,reindeer elf mistletoe gingerbread ornament ad...,905.596416
2,reindeer,reindeer scrooge elf mistletoe gingerbread adv...,938.000048
3,reindeer,reindeer advent elf gingerbread mistletoe orna...,1026.175178
4,reindeer,reindeer mistletoe elf night and chimney adven...,1247.511754
5,reindeer,reindeer chimney elf gingerbread night advent ...,1780.011315
6,reindeer,reindeer drive mistletoe gingerbread ornament ...,1773.071709
7,reindeer,reindeer mistletoe elf gingerbread chimney adv...,1026.175178
8,reindeer,reindeer chimney elf gingerbread family advent...,1516.588601
9,reindeer,reindeer jump elf gingerbread ornament advent ...,1195.043063


Processing 2th sample


  0%|          | 0/16 [00:00<?, ?it/s]

New best = 504.04031250414465 with 'jingle yuletide carol cheer grinch stocking holly naughty nice polar beard sleigh workshop chimney nutcracker ornament decorations gifts holiday magi'
New best = 494.29129035606036 with 'jingle grinch carol magi yuletide holiday holly naughty nice polar beard sleigh workshop chimney nutcracker ornament decorations gifts stocking cheer'
New best = 467.98558773246197 with 'jingle magi carol cheer grinch holiday holly naughty nice polar beard sleigh workshop decorations nutcracker ornament chimney gifts stocking yuletide'
Iteration 0, Temperature = 49.5
2th sample: jingle magi carol cheer grinch holiday holly naughty nice polar beard sleigh workshop decorations nutcracker ornament chimney gifts stocking yuletide
Elapsed time: 32.21 sec
Best value: 467.98558773246197
jingle magi carol cheer grinch holiday holly naughty nice polar beard sleigh workshop decorations nutcracker ornament chimney gifts stocking yuletide


,start,text,score
0,jingle,jingle yuletide carol cheer grinch polar magi ...,507.993550
1,jingle,jingle carol sleigh cheer grinch workshop chim...,584.696992
2,jingle,jingle cheer carol yuletide grinch holiday hol...,562.297602
3,jingle,jingle holiday carol cheer yuletide grinch nut...,642.164017
4,jingle,jingle yuletide decorations carol grinch holid...,675.616090
5,jingle,jingle holly carol yuletide grinch nutcracker ...,670.358404
6,jingle,jingle workshop carol cheer grinch holiday hol...,1092.357791
7,jingle,jingle beard carol cheer holly holiday yuletid...,768.569696
8,jingle,jingle holly carol cheer grinch holiday yuleti...,584.696992
9,jingle,jingle beard carol cheer grinch holiday holly ...,530.297145


In [13]:
sub_df

,id,text,best_value
0,0,advent chimney elf family fireplace gingerbrea...,0.000000
1,1,reindeer elf mistletoe gingerbread ornament ad...,857.401454
2,2,jingle magi carol grinch cheer nutcracker holi...,564.498372
3,3,yuletide decorations gifts cheer holiday carol...,0.000000
4,4,hohoho candle poinsettia snowglobe peppermint ...,0.000000
5,5,advent chimney elf family fireplace gingerbrea...,0.000000


In [14]:
# 各セルに関数を適用
sub_df["text"] = sub_df["text"].astype(str)
sub_df = sub_df.applymap(add_quotes_to_strings)

/tmp/ipykernel_56/428122046.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sub_df = sub_df.applymap(add_quotes_to_strings)


In [15]:
sub_df[["id", "text"]].to_csv(f"{path}/out/submission_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)
sub_df.to_csv(f"{path}/out/score_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)

In [16]:
np.mean(sub_df["best_value"])

np.float64(236.98330437117946)